In [1]:
!gdown --id 1SOIlM2PaImoHwn0WMKMviv53PGKy7ZV4

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1SOIlM2PaImoHwn0WMKMviv53PGKy7ZV4
To: /content/gaming_content.csv
100% 5.90M/5.90M [00:00<00:00, 101MB/s]


In [2]:
%pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-google-genai \
                                google-generativeai \
                                html2text \
                                chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/

In [3]:
import os

from datetime import datetime

import google.generativeai as genai

from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.docstore.document import Document

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma


In [4]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [5]:
loader = CSVLoader(file_path='/content/gaming_content.csv')
data = loader.load()

In [6]:
data

[Document(page_content=': 0\nname: Balatro\ndescription: Balatro is a poker-inspired roguelike deck builder all about creating powerful synergies and winning big.   \nCombine valid poker hands with unique Joker cards in order to create varied synergies and builds. Earn enough chips to beat devious blinds, all while uncovering hidden bonus hands and decks as you progress. You’re going to need every edge you can get in order to reach the boss blind, beat the final ante and secure victory.  The definitive poker roguelike experience.Endless possibilities - every pick-up, discard and joker can dramatically alter the course of your run.In-depth campaign mode complete with 8 difficulties, as well as challenge, daily and seeded runs.    \nDeftly deploy an array of potent tools: different Decks, Jokers, Tarot cards, Planet cards, Spectral cards, and Vouchers. Use these versatile instruments to ignite a combo frenzy and turbocharge your poker hands, crafting electrifying, adrenaline-fueled syner

In [7]:
# Follow this link to setup Gemini's embedding model: https://github.com/google/generative-ai-docs/blob/main/examples/gemini/python/langchain/Gemini_LangChain_QA_Chroma_WebLoad.ipynb

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                                 google_api_key=GEMINI_API_KEY)

vectorstore = Chroma.from_documents(
                     documents=data,                 # Data
                     embedding=gemini_embeddings,    # Embedding model
                     persist_directory="./chroma_db" # Directory to save data
                     )


In [8]:

# Load from disk
vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",       # Directory of db
                        embedding_function=gemini_embeddings   # Embedding model
                   )

retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 5})


print(len(retriever.get_relevant_documents('Deadliest game')))


5


In [9]:
llm = GoogleGenerativeAI(model="gemini-pro",
                        temperature=0.7,
                        top_p=0.85,
                        google_api_key=GEMINI_API_KEY)
llm

GoogleGenerativeAI(model='gemini-pro', google_api_key=SecretStr('**********'), top_p=0.85, client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
))

In [10]:

# Prompt template to query Gemini
llm_prompt_template = """You are an assistant for question-answering tasks.
Your task is to get relevant game based on the following context that is given by an user.
If you don't know the answer, just say that you don't know.
Use five sentences maximum and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks.\nYour task is to get relevant game based on the following context that is given by an user.\nIf you don't know the answer, just say that you don't know.\nUse five sentences maximum and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:"


In [11]:

# Combine data from documents to readable string format.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [12]:
user_description = "I want to play violent game that take place in Japan. I want to play it with my 3 friends. What game should I play?"
rag_chain.invoke(user_description)

'I recommend you to play "Shadow Tactics: Blades of the Shogun". It is a hardcore tactical stealth game set in Japan around the Edo period. You can play with up to 4 players.'